In [1]:
# %pip install tensorflow

In [14]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
# from tensorflow.keras import callbacks
 

import copy
import random
from json.encoder import INFINITY
import pandas as pd
import math
from ayo_game import play, is_illegal_move, assign_reward, print_game_play, end_game
from agents import random_agent as ra
from agents import minimax_agent as ma
from agents import mcts_agent as mctsa
seed = 37

randint = random.randint
random.seed(seed)
# checkpoint_filepath = 'latest_value_model.keras'
checkpoint_filepath = 'latest.keras'
# checkpoint_filepath = 'multi.keras'

In [19]:
# The model (that are considered the best) can be loaded as -
def predict(model, data):
        # Predict action for a new data point
        data = np.array(data)
        prediction = model.predict(data)
        return prediction[0][0]

value_model = load_model(checkpoint_filepath)
data = [
    [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 6],# 0
]


predict(value_model, data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step


-5.8536077

In [ ]:
class Node:
    def __init__(self,prior,state,parent=None):
        self.parent = parent
        self.children = {}
        self.prior = prior
        # self.action = action
        # self.legal_actions = []
        self.total_score = 0
        self.visit_count = 0
        self.expanded = False
        self.state = state

    def update_result(self, reward):
        self.total_score += reward
        self.visit_count += 1


    def __init__(self, prior, to_play):
        self.visit_count = 0
        self.to_play = to_play
        self.prior = prior
        self.value_sum = 0
        self.children = {}
        self.state = None

    def expanded(self):
        return len(self.children) > 0

    def value(self):
        if self.visit_count == 0:
            return 0
        return self.value_sum / self.visit_count



    def select_action(self, temperature):
        """
        Select action according to the visit count distribution and the temperature.
        """
        visit_counts = np.array([child.visit_count for child in self.children.values()])
        actions = [action for action in self.children.keys()]
        if temperature == 0:
            action = actions[np.argmax(visit_counts)]
        elif temperature == float("inf"):
            action = np.random.choice(actions)
        else:
            # See paper appendix Data Generation
            visit_count_distribution = visit_counts ** (1 / temperature)
            visit_count_distribution = visit_count_distribution / sum(visit_count_distribution)
            action = np.random.choice(actions, p=visit_count_distribution)

        return action
    


    def ucb_score(parent, child):
        """
        The score for an action that would transition between the parent and child.
        """
        prior_score = child.prior * math.sqrt(parent.visit_count) / (child.visit_count + 1)
        # if child.visit_count > 0:
        #     # The value of the child is from the perspective of the opposing player
        #     value_score = -child.value()
        # else:
        #     value_score = 0

        value_score = -child.value()
        return value_score + prior_score
    


    def select_child(self):
        """
        Select the child with the highest UCB score.
        """
        best_score = -np.inf
        best_action = -1
        best_child = None

        for action, child in self.children.items():
            score = ucb_score(self, child)
            if score > best_score:
                best_score = score
                best_action = action
                best_child = child

        return best_action, best_child

    def expand(self, state, action_probs):
        """
        We expand a node and keep track of the prior policy probability given by neural network
        """
        # self.state = state
        for action, prob in enumerate(action_probs):
            if prob != 0:
                new_state, _ = play(state, action)
                self.children[action] = Node(prior=prob,state=new_state, parent=self)

    def printer(self):
        print('parent_node: ',self.parent_node)
        print('action: ',self.action)
        print('legal_actions: ',self.legal_actions)
        print('total_score: ',self.total_score)
        print('visit_count: ',self.visit_count)
        print('expanded: ',self.expanded)
        print('children: ',self.children)
        print('state: ',self.state)


In [ ]:
def ucb_score(parent, child):
    """
    The score for an action that would transition between the parent and child.
    """
    prior_score = child.prior * math.sqrt(parent.visit_count) / (child.visit_count + 1)
    if child.visit_count > 0:
        # The value of the child is from the perspective of the opposing player
        value_score = -child.value()
    else:
        value_score = 0

    return value_score + prior_score




class MCTS:

    def __init__(self, game, model, args):
        self.game = game
        self.model = model
        self.args = args

    def run(self, model, state, to_play):

        root = Node(0, to_play)

        # EXPAND root
        action_probs, value = model.predict(state)
        valid_moves = self.game.get_valid_moves(state)
        action_probs = action_probs * valid_moves  # mask invalid moves
        action_probs /= np.sum(action_probs)
        root.expand(state, to_play, action_probs)

        for _ in range(self.args['num_simulations']):
            node = root
            search_path = [node]

            # SELECT
            while node.expanded():
                action, node = node.select_child()
                search_path.append(node)

            parent = search_path[-2]
            state = parent.state
            next_state, reward = play(state, action=action)
            if value is None:
                # If the game has not ended:
                # EXPAND
                action_probs, value = model.predict(next_state)
                valid_moves = self.game.get_valid_moves(next_state)
                action_probs = action_probs * valid_moves  # mask invalid moves
                action_probs /= np.sum(action_probs)
                node.expand(next_state, parent.to_play * -1, action_probs)

            self.backpropagate(search_path, value, parent.to_play * -1)

        return root

    def backpropagate(self, search_path, value, to_play):
        """
        At the end of a simulation, we propagate the evaluation all the way up the tree
        to the root.
        """
        for node in reversed(search_path):
            node.value_sum += value if node.to_play == to_play else -value
            node.visit_count += 1

In [2]:
random_agent = ra.agent
minimax_agent = ma.agent
minimax_value = ma.get_action_value

In [153]:
def format_state(state):
    x = []
    x.extend(state['board'])
    x.append(state['current_player'])
    x.append(state['player_territory'][1])

    return x

In [154]:

def remove_duplicates(arr):
    # Convert the list of lists to a NumPy array
    np_arr = np.array(arr, dtype=object)
    
    # Create a set to keep track of unique first elements (list of numbers)
    seen = set()
    unique_arr = []

    for item in np_arr:
        # Convert the list of numbers (first element) to a tuple so it can be added to a set
        num_tuple = tuple(item[0])
        if num_tuple not in seen:
            unique_arr.append(item)
            seen.add(num_tuple)
    
    return np.array(unique_arr, dtype=object)

In [155]:
def generate_action(state,agent_1, agent_2):
   if state['current_player'] == 0:
      func = agent_1['func']
      arg = agent_1['arg']
      return func(state, arg)

   if state['current_player'] == 1:
      func = agent_2['func']
      arg = agent_2['arg']
      return func(state, arg)

In [156]:
def execute_episode(state,  agent_1, agent_2,show=False):
    state = copy.deepcopy(state)
    reward = [0,0,0]
    path = []
    train_example_policy = []
    train_example_value = []
    _, value = minimax_value(state, agent_2['arg'])
    train_example_value.append([format_state(state), value])

    while True:
        action = generate_action(state, agent_1, agent_2)
        # print([format_state(state), value])
        
        if is_illegal_move(state, action):
            continue

        state, new_reward = play(state, action)
        _, value = minimax_value(state, agent_2['arg'])
        train_example_value.append([format_state(state), value])
        reward = assign_reward(reward, new_reward)
        if show:
            print_game_play(state, reward, action)

        if end_game(state):
            break
        path.append(action)

    return  train_example_value

In [167]:
def lol():
    num_of_eps = 1000
    data = []
    state = {
   # 'board' :[6, 6, 2, 7, 1, 6, 1, 6, 6, 6, 0, 1],
   'board' :[4,4,4,4,4,4,4,4,4,4,4,4],
   'current_player': 0,
   'player_territory': (0,6)
    }
    
    agent_1 = {
    'func': random_agent,
    'arg': {},
    'name': 'random_agent',
    'elo': 1200
    }
        
    agent_2 = {
    'func': minimax_agent,
    'arg': {
        'max_dept': 9,
    },
    'name': 'minimax_agent_dept_3',
    'elo': 1200
    }

    for i in range(num_of_eps):
        state['current_player'] = i%2
        training_examples = execute_episode(state,  agent_1, agent_2)
        print(training_examples)
        data.extend(training_examples)
        print((i/num_of_eps) * 100)
    unique_data = remove_duplicates(data)
    print(len(unique_data))
    value_model.train(data)

In [ ]:
lol()

[[[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 6], 0], [[6, 0, 1, 6, 6, 2, 7, 1, 6, 1, 6, 6, 1, 6], -12], [[10, 0, 0, 1, 9, 2, 10, 0, 0, 3, 0, 1, 0, 6], -8], [[11, 1, 0, 1, 0, 3, 11, 1, 1, 0, 1, 2, 1, 6], -16], [[12, 0, 1, 0, 1, 0, 11, 1, 1, 0, 1, 0, 0, 6], -4], [[12, 0, 1, 0, 0, 1, 11, 1, 1, 0, 1, 0, 1, 6], -12], [[0, 1, 0, 0, 2, 0, 13, 2, 0, 3, 0, 3, 0, 6], -8], [[0, 3, 0, 2, 0, 3, 0, 1, 1, 1, 0, 1, 1, 6], 4], [[1, 0, 0, 2, 0, 3, 0, 1, 0, 0, 1, 0, 0, 6], 8], [[1, 0, 0, 2, 0, 0, 1, 2, 1, 0, 1, 0, 1, 6], 0], [[0, 1, 1, 2, 0, 0, 0, 0, 2, 1, 0, 1, 0, 6], 0], [[0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0, 1, 1, 6], 0], [[0, 0, 0, 3, 1, 0, 0, 0, 2, 0, 1, 1, 0, 6], 0], [[0, 0, 0, 0, 2, 1, 1, 0, 2, 0, 1, 1, 1, 6], -8], [[0, 0, 0, 0, 2, 1, 0, 1, 2, 0, 1, 1, 0, 6], -8], [[0, 0, 0, 0, 2, 0, 1, 1, 2, 0, 1, 1, 1, 6], -8], [[0, 0, 0, 0, 2, 0, 0, 0, 3, 1, 1, 1, 0, 6], -8], [[0, 0, 0, 0, 0, 1, 1, 0, 3, 1, 1, 1, 1, 6], -8], [[0, 0, 0, 0, 0, 1, 0, 1, 3, 1, 1, 1, 0, 6], -8], [[0, 0, 0, 0, 0, 0, 1, 1, 3, 1, 1, 1, 1, 6],

In [160]:
# play games
# store outcome of games
# train model
data = [
    [6, 2, 7, 1, 6, 1, 6, 6, 6, 0, 1, 6, 1, 6],# -8
    [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 6],# 0
    [3, 1, 3, 0, 2, 1, 0, 2, 2, 0, 2, 0, 0, 6],# 0
    [0, 0, 0, 0, 1, 1, 0, 1, 3, 1, 0, 1, 1, 6],# -8
]
# data = [[0, 0, 3, 11, 1, 1, 0, 1, 2, 11, 1, 1, 1, 6]]
value_model.predict(data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


array([[-0.68214655],
       [-0.56537616],
       [-0.68339497],
       [-0.882059  ],
       [-0.7181479 ],
       [-1.4718579 ],
       [-0.9175086 ],
       [-0.9333542 ]], dtype=float32)

In [ ]:
checkpoint_filepath = 'checkpoint.model.keras'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

# Model is saved at the end of every epoch, if it's the best seen so far.
model.fit(epochs=EPOCHS, callbacks=[model_checkpoint_callback])

# The model (that are considered the best) can be loaded as -
keras.models.load_model(checkpoint_filepath)

# Alternatively, one could checkpoint just the model weights as -
checkpoint_filepath = '/tmp/ckpt/checkpoint.weights.h5'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

# Model weights are saved at the end of every epoch, if it's the best seen
# so far.
model.fit(epochs=EPOCHS, callbacks=[model_checkpoint_callback])

# The model weights (that are considered the best) can be loaded as -
model.load_weights(checkpoint_filepath)